# Single Source Shortest Path
The _Single Source Shortest Path_ (SSSP) algorithm calculates the shortest (weighted) path between a pair of nodes.
Dijkstra's algorithm is the most well known one in this category.
SSSP is a real time graph algorithm - it can be used as part of the normal user flow in a web or mobile application.

First we'll import the Neo4j driver and Pandas libraries:


In [1]:
from neo4j import GraphDatabase
import pandas as pd
import os

Next let's create an instance of the Neo4j driver which we'll use to execute our queries.


In [2]:
host = os.environ.get("NEO4J_HOST", "bolt://localhost") 
user = os.environ.get("NEO4J_USER", "neo4j")
password = os.environ.get("NEO4J_PASSWORD", "neo")
driver = GraphDatabase.driver(host, auth=(user, password))

Now let's create a sample graph that we'll run the algorithm against.


In [3]:
create_graph_query = '''
CREATE (a:Loc{name:'A'}), (b:Loc{name:'B'}), (c:Loc{name:'C'}), 
       (d:Loc{name:'D'}), (e:Loc{name:'E'}), (f:Loc{name:'F'}),
       (a)-[:ROAD {cost:50}]->(b),
       (a)-[:ROAD {cost:50}]->(c),
       (a)-[:ROAD {cost:100}]->(d),
       (a)-[:RAIL {cost:50}]->(d),
       (b)-[:ROAD {cost:40}]->(d),
       (c)-[:ROAD {cost:40}]->(d),
       (c)-[:ROAD {cost:80}]->(e),
       (d)-[:ROAD {cost:30}]->(e),
       (d)-[:ROAD {cost:80}]->(f),
       (e)-[:ROAD {cost:40}]->(f),
       (e)-[:RAIL {cost:20}]->(f);
'''

with driver.session() as session:
    result = session.write_transaction(lambda tx: tx.run(create_graph_query))
    print("Stats: " + str(result.consume().metadata.get("stats", {})))

Stats: {'labels-added': 6, 'relationships-created': 11, 'nodes-created': 6, 'properties-set': 17}


Finally we can run the algorithm by executing the following query:


In [4]:
streaming_query = """
MATCH (start:Loc {name: 'A'}), (end:Loc {name: 'F'})
CALL gds.alpha.shortestPath.stream({
  nodeProjection: 'Loc',
  relationshipProjection: {
    ROAD: {
      type: '*',
      properties: 'cost',
      orientation: 'UNDIRECTED'
    }
  },
  startNode: start,
  endNode: end,
  relationshipWeightProperty: 'cost'
})
YIELD nodeId, cost
RETURN gds.util.asNode(nodeId).name AS name, cost
"""

with driver.session() as session:
    result = session.run(streaming_query)
    df = pd.DataFrame([dict(r) for r in result])

df

name   cost
0    A    0.0
1    D   50.0
2    E   80.0
3    F  100.0

The quickest route takes us from "A" to "F" via "D" and "E" at a total cost of 100.
We first go by rail from "A" to "D" at a cost of 50, from "D" to "E" by road for an additional 30, and finally from "E" to "F" by rail for an additional 20.